In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic-app').getOrCreate()

In [3]:
df = spark.read.csv('course-material/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv', header=True, inferSchema=True)

In [4]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = df.select(['Survived','Pclass', 'Sex', 'Age','SibSp', 'Parch', 'Fare', 'Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,OneHotEncoder, StringIndexer)

In [17]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
# A B C
# 0 1 2
# ONE HOT ENCODE
# KEY A B C
# Example A
# [1, 0, 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [18]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [19]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'], outputCol='features')

In [20]:
from pyspark.ml.classification import LogisticRegression

In [21]:
from pyspark.ml import Pipeline

In [22]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [23]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [24]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [25]:
fit_model = pipeline.fit(train_data)

In [30]:
results = fit_model.transform(test_data)

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [32]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [47]:
type(results)

pyspark.sql.dataframe.DataFrame

In [42]:
result.select('Survived', 'prediction', 'probability', 'Sex', 'Age').show()

+--------+----------+--------------------+------+----+
|Survived|prediction|         probability|   Sex| Age|
+--------+----------+--------------------+------+----+
|       0|       1.0|[0.06046627418010...|female|50.0|
|       0|       1.0|[0.49009732610223...|  male|19.0|
|       0|       1.0|[0.26478546123120...|  male|24.0|
|       0|       1.0|[0.37526334903760...|  male|28.0|
|       0|       0.0|[0.51389180956782...|  male|31.0|
|       0|       1.0|[0.40698931507098...|  male|37.0|
|       0|       0.0|[0.55325795734450...|  male|38.0|
|       0|       0.0|[0.62067658793819...|  male|45.0|
|       0|       1.0|[0.44166764946970...|  male|46.0|
|       0|       0.0|[0.53012987931618...|  male|50.0|
|       0|       0.0|[0.72847335523137...|  male|52.0|
|       0|       0.0|[0.68618451322098...|  male|54.0|
|       0|       0.0|[0.69684724550174...|  male|56.0|
|       0|       0.0|[0.53045336478137...|  male|56.0|
|       0|       0.0|[0.54562899829514...|  male|58.0|
|       0|

In [34]:
AUC = my_eval.evaluate(results)

In [35]:
AUC

0.7688891157851746